# Section 0: Import Libraries

In [185]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import requests
import time
from sqlalchemy import create_engine, func
from sqlalchemy.ext.automap import automap_base
import pymongo
from sqlalchemy.orm import Session
from config import gkey
%matplotlib inline

# Section 1: Load CSVs

In [186]:
#Banzuke file (fighters)
banzuke = pd.read_csv('../Resources/banzuke.csv')

In [187]:
banzuke.head()

,basho,id,rank,rikishi,heya,shusshin,birth_date,height,weight,prev,prev_w,prev_l
0,1983.01,1354,Y1e,Chiyonofuji,Kokonoe,Hokkaido,1955-06-01,182.0,116.0,Y1e,14.0,1.0
1,1983.01,4080,Y1w,Kitanoumi,Mihogaseki,Hokkaido,1953-05-16,179.0,165.0,Y2eHD,9.0,3.0
2,1983.01,4095,Y2eHD,Wakanohana,Futagoyama,Aomori,1953-04-03,186.0,133.0,Y1w,0.0,0.0
3,1983.01,4104,O1e,Takanosato,Futagoyama,Aomori,1952-09-29,181.0,144.0,O1e,10.0,5.0
4,1983.01,4112,O1w,Kotokaze,Sadogatake,Mie,1957-04-26,183.0,163.0,O1w,10.0,5.0


In [188]:
banzuke['id'].value_counts()

271      209
607      184
136      173
82       170
179      168
        ... 
7761       1
2803       1
972        1
2908       1
12178      1
Name: id, Length: 5158, dtype: int64

In [189]:
banzuke[banzuke['id']==209].tail()

,basho,id,rank,rikishi,heya,shusshin,birth_date,height,weight,prev,prev_w,prev_l
76920,1998.11,209,Ms46w,Asaota,Wakamatsu,Aomori,1971-06-19,186.0,140.0,Ms37w,3.0,4.0
77752,1999.01,209,Sd7w,Asaota,Wakamatsu,Aomori,1971-06-19,186.0,140.0,Ms46w,2.0,5.0
78580,1999.03,209,Sd33e,Asaota,Wakamatsu,Aomori,1971-06-19,186.0,136.5,Sd7w,2.0,5.0
79308,1999.05,209,Sd19e,Asaota,Wakamatsu,Aomori,1971-06-19,186.0,136.5,Sd33e,4.0,3.0
80128,1999.07,209,Sd6w,Asaota,Wakamatsu,Aomori,1971-06-19,186.0,136.5,Sd19e,4.0,3.0


In [190]:
#Results file
results = pd.read_csv('../Resources/results.csv')
results.head()

,basho,day,rikishi1_id,rikishi1_rank,rikishi1_shikona,rikishi1_result,rikishi1_win,kimarite,rikishi2_id,rikishi2_rank,rikishi2_shikona,rikishi2_result,rikishi2_win
0,1983.01,1,4140,J13w,Chikubayama,0-1 (7-8),0,yorikiri,4306,Ms1e,Ofuji,1-0 (6-1),1
1,1983.01,1,4306,Ms1e,Ofuji,1-0 (6-1),1,yorikiri,4140,J13w,Chikubayama,0-1 (7-8),0
2,1983.01,1,1337,J12w,Tochitsukasa,1-0 (9-6),1,oshidashi,4323,J13e,Shiraiwa,0-1 (3-12),0
3,1983.01,1,4323,J13e,Shiraiwa,0-1 (3-12),0,oshidashi,1337,J12w,Tochitsukasa,1-0 (9-6),1
4,1983.01,1,4097,J12e,Tamakiyama,0-1 (8-7),0,yorikiri,4319,J11w,Harunafuji,1-0 (5-10),1


In [191]:
#Odds file
odds = pd.read_csv('../Resources/odds.csv')
odds.head()

,rikishi1,odds1,rikishi2,odds2,ts
0,Arawashi,1.8,Ura,2.01,2017-05-13 09:00:01
1,Daishomaru,1.9,Onosho,1.90,2017-05-13 09:00:01
2,Goeido,1.3,Okinoumi,3.52,2017-05-13 09:00:01
3,Hakuho,1.1,Chiyonokuni,7.00,2017-05-13 09:00:01
4,Harumafuji,1.4,Kotoshogiku,2.96,2017-05-13 09:00:01


# Section 2: Limit Data to Focus on Top Division Only

In [192]:
topdiv = banzuke[(banzuke['rank'].str.contains('Y\d'))|(banzuke['rank'].str.contains('M\d'))|(banzuke['rank'].str.contains('K\d'))|(banzuke['rank'].str.contains('S\d'))|(banzuke['rank'].str.startswith('O'))]

In [193]:
topdiv.head()

,basho,id,rank,rikishi,heya,shusshin,birth_date,height,weight,prev,prev_w,prev_l
0,1983.01,1354,Y1e,Chiyonofuji,Kokonoe,Hokkaido,1955-06-01,182.0,116.0,Y1e,14.0,1.0
1,1983.01,4080,Y1w,Kitanoumi,Mihogaseki,Hokkaido,1953-05-16,179.0,165.0,Y2eHD,9.0,3.0
2,1983.01,4095,Y2eHD,Wakanohana,Futagoyama,Aomori,1953-04-03,186.0,133.0,Y1w,0.0,0.0
3,1983.01,4104,O1e,Takanosato,Futagoyama,Aomori,1952-09-29,181.0,144.0,O1e,10.0,5.0
4,1983.01,4112,O1w,Kotokaze,Sadogatake,Mie,1957-04-26,183.0,163.0,O1w,10.0,5.0


In [194]:
df.info(), topdiv.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 169764 entries, 0 to 169763
Data columns (total 12 columns):
 #   Column      Non-Null Count   Dtype  
---  ------      --------------   -----  
 0   basho       169764 non-null  float64
 1   id          169764 non-null  int64  
 2   rank        169764 non-null  object 
 3   rikishi     169764 non-null  object 
 4   heya        169764 non-null  object 
 5   shusshin    169764 non-null  object 
 6   birth_date  169101 non-null  object 
 7   height      164590 non-null  float64
 8   weight      164590 non-null  float64
 9   prev        169685 non-null  object 
 10  prev_w      169685 non-null  float64
 11  prev_l      169685 non-null  float64
dtypes: float64(5), int64(1), object(6)
memory usage: 15.5+ MB
<class 'pandas.core.frame.DataFrame'>
Int64Index: 9293 entries, 0 to 169146
Data columns (total 12 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   basho       9293 non-null   float64
 1   i

(None, None)

In [195]:
#Get unique fighters in Top Division
fighters = topdiv.copy()
fighters.drop_duplicates(subset='id',keep='last', inplace=True)

In [196]:
fighters[fighters['rikishi']=='Terunofuji']

,basho,id,rank,rikishi,heya,shusshin,birth_date,height,weight,prev,prev_w,prev_l
169106,2021.07,11927,O1e,Terunofuji,Isegahama,Mongolia,1991-11-29,191.0,173.0,O2w,12.0,3.0


In [197]:
#Drop not needed columns in fighters dataset
fighters.drop(['basho','prev','prev_w','prev_l'],axis=1,inplace=True)

In [198]:
fighters.reset_index(drop=True, inplace=True)

In [199]:
fighters.head()

,id,rank,rikishi,heya,shusshin,birth_date,height,weight
0,4095,Y2eHD,Wakanohana,Futagoyama,Aomori,1953-04-03,186.0,133.0
1,4122,M14e,Kurosegawa,Isegahama,Tokyo,1951-05-13,184.0,130.0
2,4103,M14w,Banryuyama,Mihogaseki,Hyogo,1951-05-04,177.5,135.0
3,4050,M8e,Takamiyama,Takasago,U.S.A.,1944-06-16,192.0,205.0
4,4077,M10w,Fujizakura,Takasago,Yamanashi,1948-02-09,180.0,139.0


In [200]:
fighters = fighters.sort_values(by=['id'])
fighters.reset_index(drop=True, inplace=True)
fighters.head()

,id,rank,rikishi,heya,shusshin,birth_date,height,weight
0,1,Y1e,Akebono,Azumazeki,U.S.A.,1969-05-08,203.5,230.5
1,2,Y1w,Takanohana,Futagoyama,Tokyo,1972-08-12,185.0,161.0
2,3,Y2w,Wakanohana,Futagoyama,Tokyo,1971-01-20,178.5,124.5
3,4,Y1w,Musashimaru,Musashigawa,U.S.A.,1971-05-02,192.0,231.5
4,5,M13e,Takanonami,Takanohana,Aomori,1971-10-27,196.0,163.5


In [201]:
#Save to CSV
fighters.to_csv('fighters.csv',index=False)

# Section 3: Heya API Calls

In [202]:
#Make copy of fighters dataset to Add stable info on Lat and Lng
stables = fighters.copy()

In [203]:
#Create needed columns to store data for each stable
stables['lat'] =''
stables['lng'] =''

#Perform API call to get information for each stable
base_url = 'https://maps.googleapis.com/maps/api/geocode/json?'


#Parameter for the API call
params = {'key':gkey}

#Iterate through the list of stables to retrieve their information
for i, row in stables.iterrows():
    
    params['address'] = row['heya']+ ' Japan'

    response = requests.get(base_url,params=params).json()
    
    
    
    try:
        stables.loc[i,'lat']= response['results'][0]['geometry']['location']['lat']
        stables.loc[i,'lng']= response['results'][0]['geometry']['location']['lng']
    except:
        #Assign NaN to Cities not found, for misisng values to be filled with closest location
        stables.loc[i,['lat','lng']]= np.nan

In [204]:
stables.head()

,id,rank,rikishi,heya,shusshin,birth_date,height,weight,lat,lng
0,1,Y1e,Akebono,Azumazeki,U.S.A.,1969-05-08,203.5,230.5,35.7585,139.868
1,2,Y1w,Takanohana,Futagoyama,Tokyo,1972-08-12,185.0,161.0,43.0431,141.316
2,3,Y2w,Wakanohana,Futagoyama,Tokyo,1971-01-20,178.5,124.5,43.0431,141.316
3,4,Y1w,Musashimaru,Musashigawa,U.S.A.,1971-05-02,192.0,231.5,35.7128,139.864
4,5,M13e,Takanonami,Takanohana,Aomori,1971-10-27,196.0,163.5,NaN,NaN


In [205]:
#Drop not needed columns in stables dataset
stables.drop(['id','rank','rikishi','shusshin', 'birth_date', 'height', 'weight'],axis=1,inplace=True)
stables.head()

,heya,lat,lng
0,Azumazeki,35.7585,139.868
1,Futagoyama,43.0431,141.316
2,Futagoyama,43.0431,141.316
3,Musashigawa,35.7128,139.864
4,Takanohana,NaN,NaN


In [206]:
#Fill out Nan values with previous heya
stables.fillna(method='ffill',inplace=True)
stables.head()

,heya,lat,lng
0,Azumazeki,35.758530,139.868003
1,Futagoyama,43.043111,141.316019
2,Futagoyama,43.043111,141.316019
3,Musashigawa,35.712773,139.864346
4,Takanohana,35.712773,139.864346


In [207]:
stables.describe()

,lat,lng
count,332.000000,332.000000
mean,36.218955,127.279049
std,3.353838,48.204278
min,26.533469,-122.706879
25%,34.766101,135.125987
50%,35.693738,139.387303
75%,37.152033,139.846956
max,45.519408,142.480038


In [208]:
#Identify unique stables by removing duplicates
stables.drop_duplicates(subset='heya',inplace=True)
stables.head()

,heya,lat,lng
0,Azumazeki,35.758530,139.868003
1,Futagoyama,43.043111,141.316019
3,Musashigawa,35.712773,139.864346
4,Takanohana,35.712773,139.864346
6,Tomozuna,35.705972,139.813157


In [209]:
stables.reset_index(drop=True, inplace=True)
stables.head()

,heya,lat,lng
0,Azumazeki,35.758530,139.868003
1,Futagoyama,43.043111,141.316019
2,Musashigawa,35.712773,139.864346
3,Takanohana,35.712773,139.864346
4,Tomozuna,35.705972,139.813157


In [210]:
stables.describe()

,lat,lng
count,57.000000,57.000000
mean,36.101846,121.143710
std,2.994016,60.878023
min,26.533469,-122.706879
25%,34.740884,134.997586
50%,35.692360,138.745596
75%,36.326032,139.814420
max,45.519408,142.480038


In [211]:
stables.insert(0, 'stable_id', range(0,57))
stables.head()

,stable_id,heya,lat,lng
0,0,Azumazeki,35.758530,139.868003
1,1,Futagoyama,43.043111,141.316019
2,2,Musashigawa,35.712773,139.864346
3,3,Takanohana,35.712773,139.864346
4,4,Tomozuna,35.705972,139.813157


range(1, 57)

In [212]:
stables.tail()

,stable_id,heya,lat,lng
52,52,Onoe,38.872941,-77.414637
53,53,Shikoroyama,35.589152,139.285410
54,54,Kise,26.533469,127.941375
55,55,Chiganoura,35.724050,139.806394
56,56,Tokiwayama,34.075673,131.825297


In [213]:
#Create dictionary of Stables
temp_dict = pd.Series(stables.heya,index=stables.stable_id).to_dict()
stable_to_id_dict = dict([(value, key) for key, value in temp_dict.items()])

In [214]:
stable_to_id_dict

{'Azumazeki': 0,
 'Futagoyama': 1,
 'Musashigawa': 2,
 'Takanohana': 3,
 'Tomozuna': 4,
 'Kokonoe': 5,
 'Kasugano': 6,
 'Tokitsukaze': 7,
 'Sadogatake': 8,
 'Isenoumi': 9,
 'Dewanoumi': 10,
 'Minato': 11,
 'Kataonami': 12,
 'Tamanoi': 13,
 'Tatsutagawa': 14,
 'Oshima': 15,
 'Magaki': 16,
 'Kitanoumi': 17,
 'Mihogaseki': 18,
 'Takasago': 19,
 'Izutsu': 20,
 'Wakamatsu': 21,
 'Tagonoura': 22,
 'Hakkaku': 23,
 'Tatsunami': 24,
 'Nishonoseki': 25,
 'Oshiogawa': 26,
 'Irumagawa': 27,
 'Naruto': 28,
 'Asahiyama': 29,
 'Michinoku': 30,
 'Matsugane': 31,
 'Isegahama': 32,
 'Hanaregoma': 33,
 'Ajigawa': 34,
 'Oitekaze': 35,
 'Miyagino': 36,
 'Oguruma': 37,
 'Yamahibiki': 38,
 'Fujishima': 39,
 'Kasugayama': 40,
 'Onomatsu': 41,
 'Sakaigawa': 42,
 'Hanakago': 43,
 'Takadagawa': 44,
 'Kagamiyama': 45,
 'Taiho': 46,
 'Onaruto': 47,
 'Kumagatani': 48,
 'Otake': 49,
 'Minezaki': 50,
 'Arashio': 51,
 'Onoe': 52,
 'Shikoroyama': 53,
 'Kise': 54,
 'Chiganoura': 55,
 'Tokiwayama': 56}

In [215]:
#Add stable id to fighters dataset
fighters.head()

,id,rank,rikishi,heya,shusshin,birth_date,height,weight
0,1,Y1e,Akebono,Azumazeki,U.S.A.,1969-05-08,203.5,230.5
1,2,Y1w,Takanohana,Futagoyama,Tokyo,1972-08-12,185.0,161.0
2,3,Y2w,Wakanohana,Futagoyama,Tokyo,1971-01-20,178.5,124.5
3,4,Y1w,Musashimaru,Musashigawa,U.S.A.,1971-05-02,192.0,231.5
4,5,M13e,Takanonami,Takanohana,Aomori,1971-10-27,196.0,163.5


In [216]:
fighters['stable_id'] = fighters['heya'].map(stable_to_id_dict)

In [217]:
fighters.head()

,id,rank,rikishi,heya,shusshin,birth_date,height,weight,stable_id
0,1,Y1e,Akebono,Azumazeki,U.S.A.,1969-05-08,203.5,230.5,0
1,2,Y1w,Takanohana,Futagoyama,Tokyo,1972-08-12,185.0,161.0,1
2,3,Y2w,Wakanohana,Futagoyama,Tokyo,1971-01-20,178.5,124.5,1
3,4,Y1w,Musashimaru,Musashigawa,U.S.A.,1971-05-02,192.0,231.5,2
4,5,M13e,Takanonami,Takanohana,Aomori,1971-10-27,196.0,163.5,3


In [218]:
#Drop not needed columns in fighters dataset
fighters.drop(['heya'],axis=1,inplace=True)
fighters.head()

,id,rank,rikishi,shusshin,birth_date,height,weight,stable_id
0,1,Y1e,Akebono,U.S.A.,1969-05-08,203.5,230.5,0
1,2,Y1w,Takanohana,Tokyo,1972-08-12,185.0,161.0,1
2,3,Y2w,Wakanohana,Tokyo,1971-01-20,178.5,124.5,1
3,4,Y1w,Musashimaru,U.S.A.,1971-05-02,192.0,231.5,2
4,5,M13e,Takanonami,Aomori,1971-10-27,196.0,163.5,3


In [220]:
#Reorganize columns
fighters = fighters[['id','stable_id','rank','rikishi','shusshin','birth_date','height','weight']]
fighters.head()

,id,stable_id,rank,rikishi,shusshin,birth_date,height,weight
0,1,0,Y1e,Akebono,U.S.A.,1969-05-08,203.5,230.5
1,2,1,Y1w,Takanohana,Tokyo,1972-08-12,185.0,161.0
2,3,1,Y2w,Wakanohana,Tokyo,1971-01-20,178.5,124.5
3,4,2,Y1w,Musashimaru,U.S.A.,1971-05-02,192.0,231.5
4,5,3,M13e,Takanonami,Aomori,1971-10-27,196.0,163.5


In [221]:
stables.head()

,stable_id,heya,lat,lng
0,0,Azumazeki,35.758530,139.868003
1,1,Futagoyama,43.043111,141.316019
2,2,Musashigawa,35.712773,139.864346
3,3,Takanohana,35.712773,139.864346
4,4,Tomozuna,35.705972,139.813157


In [222]:
#Save CSVs
fighters.to_csv('fighters.csv',index=False)
stables.to_csv('stables.csv', index=False)

# Section 4: Clean Results CSV

In [223]:
results.describe()

,basho,day,rikishi1_id,rikishi1_win,rikishi2_id,rikishi2_win
count,220592.000000,220592.000000,220592.000000,220592.000000,220592.000000,220592.000000
mean,2001.754581,7.977751,2715.450071,0.500000,2715.450071,0.500000
std,10.934072,4.331359,3702.396536,0.500001,3702.396536,0.500001
min,1983.010000,1.000000,1.000000,0.000000,1.000000,0.000000
25%,1992.090000,4.000000,62.000000,0.000000,62.000000,0.000000
50%,2002.050000,8.000000,1285.000000,0.500000,1285.000000,0.500000
75%,2011.090000,12.000000,4127.000000,1.000000,4127.000000,1.000000
max,2021.070000,16.000000,12470.000000,1.000000,12470.000000,1.000000


In [224]:
results.head()

,basho,day,rikishi1_id,rikishi1_rank,rikishi1_shikona,rikishi1_result,rikishi1_win,kimarite,rikishi2_id,rikishi2_rank,rikishi2_shikona,rikishi2_result,rikishi2_win
0,1983.01,1,4140,J13w,Chikubayama,0-1 (7-8),0,yorikiri,4306,Ms1e,Ofuji,1-0 (6-1),1
1,1983.01,1,4306,Ms1e,Ofuji,1-0 (6-1),1,yorikiri,4140,J13w,Chikubayama,0-1 (7-8),0
2,1983.01,1,1337,J12w,Tochitsukasa,1-0 (9-6),1,oshidashi,4323,J13e,Shiraiwa,0-1 (3-12),0
3,1983.01,1,4323,J13e,Shiraiwa,0-1 (3-12),0,oshidashi,1337,J12w,Tochitsukasa,1-0 (9-6),1
4,1983.01,1,4097,J12e,Tamakiyama,0-1 (8-7),0,yorikiri,4319,J11w,Harunafuji,1-0 (5-10),1


In [226]:
fighters['id']

0          1
1          2
2          3
3          4
4          5
       ...  
327    12412
328    12425
329    12427
330    12449
331    12451
Name: id, Length: 332, dtype: int64

In [229]:
results[results['rikishi1_id'].isin(fighters['id'])]

,basho,day,rikishi1_id,rikishi1_rank,rikishi1_shikona,rikishi1_result,rikishi1_win,kimarite,rikishi2_id,rikishi2_rank,rikishi2_shikona,rikishi2_result,rikishi2_win
0,1983.01,1,4140,J13w,Chikubayama,0-1 (7-8),0,yorikiri,4306,Ms1e,Ofuji,1-0 (6-1),1
2,1983.01,1,1337,J12w,Tochitsukasa,1-0 (9-6),1,oshidashi,4323,J13e,Shiraiwa,0-1 (3-12),0
9,1983.01,1,4130,J9w,Hachiya,1-0 (8-7),1,hatakikomi,4111,J10e,Zaonishiki,0-1 (4-11),0
10,1983.01,1,1302,J9e,Takamisugi,1-0 (5-10),1,oshidashi,4318,J8w,Tengoyama,0-1 (7-8),0
12,1983.01,1,1361,J8e,Daitetsu,0-1 (9-6),0,yoritaoshi,4316,J7w,Kakureizan,1-0 (9-6),1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
220587,2021.07,15,12370,K1e,Wakatakakage,5-10,1,uwatenage,12210,S1w,Mitakeumi,8-7,0
220588,2021.07,15,6480,S1e,Takayasu,7-6-2,0,okuridashi,12130,O2e,Shodai,8-7,1
220589,2021.07,15,12130,O2e,Shodai,8-7,1,okuridashi,6480,S1e,Takayasu,7-6-2,0
220590,2021.07,15,1123,Y1e,Hakuho,15-0,1,kotenage,11927,O1e,Terunofuji,14-1,0


In [235]:
fighters[fighters['id']==4140]

,id,stable_id,rank,rikishi,shusshin,birth_date,height,weight
236,4140,36,M14e,Chikubayama,Fukuoka,1957-08-21,176.0,121.0


In [247]:
basho = results[(results['basho']>=2020.01) &(results['rikishi1_id'].isin(fighters['id'])) &(results['rikishi2_id'].isin(fighters['id'])) ]
basho.reset_index(drop=True, inplace=True)
basho.head()

,basho,day,rikishi1_id,rikishi1_rank,rikishi1_shikona,rikishi1_result,rikishi1_win,kimarite,rikishi2_id,rikishi2_rank,rikishi2_shikona,rikishi2_result,rikishi2_win
0,2020.01,1,11726,M17w,Tokushoryu,1-0 (14-1),1,yorikiri,11785,J1w,Chiyoshoma,0-1 (7-8),0
1,2020.01,1,11785,J1w,Chiyoshoma,0-1 (7-8),0,yorikiri,11726,M17w,Tokushoryu,1-0 (14-1),1
2,2020.01,1,6753,M16w,Kaisei,0-1 (8-7),0,hikiotoshi,12231,M17e,Kiribayama,1-0 (11-4),1
3,2020.01,1,12231,M17e,Kiribayama,1-0 (11-4),1,hikiotoshi,6753,M16w,Kaisei,0-1 (8-7),0
4,2020.01,1,6473,M16e,Tochiozan,1-0 (9-6),1,tsukiotoshi,6491,M15w,Ikioi,0-1 (8-7),0


In [248]:
basho.to_csv('basho.csv', index=False)

In [259]:
updated_fighters = fighters[fighters['id'].isin(basho['rikishi1_id'])]

In [260]:
len(updated_fighters)

63

In [261]:
basho['rikishi1_id'].nunique()

63

In [263]:
updated_fighters.reset_index(drop=True, inplace=True)
updated_fighters.head()

,id,stable_id,rank,rikishi,shusshin,birth_date,height,weight
0,1123,36,Y1e,Hakuho,Mongolia,1985-03-11,192.0,151.0
1,1219,30,Y1w,Kakuryu,Mongolia,1985-08-10,186.0,150.0
2,1226,8,M11w,Kotoshogiku,Fukuoka,1984-01-30,180.0,177.0
3,2879,42,M17e,Sadanoumi,Kumamoto,1987-05-11,184.0,131.2
4,5944,12,M10e,Tamawashi,Mongolia,1984-11-16,189.0,168.0


In [264]:
updated_fighters.to_csv('fighters.csv',index=False)

In [270]:
basho.columns

Index(['basho', 'day', 'rikishi1_id', 'rikishi1_rank', 'rikishi1_shikona',
       'rikishi1_result', 'rikishi1_win', 'kimarite', 'rikishi2_id',
       'rikishi2_rank', 'rikishi2_shikona', 'rikishi2_result', 'rikishi2_win'],
      dtype='object')

In [273]:
basho = basho[['basho', 'day', 'rikishi1_id', 'rikishi1_rank',
       'rikishi1_result', 'rikishi1_win', 'kimarite', 'rikishi2_id',
       'rikishi2_rank', 'rikishi2_result', 'rikishi2_win']]

In [274]:
basho.head()

,basho,day,rikishi1_id,rikishi1_rank,rikishi1_result,rikishi1_win,kimarite,rikishi2_id,rikishi2_rank,rikishi2_result,rikishi2_win
0,2020.01,1,11726,M17w,1-0 (14-1),1,yorikiri,11785,J1w,0-1 (7-8),0
1,2020.01,1,11785,J1w,0-1 (7-8),0,yorikiri,11726,M17w,1-0 (14-1),1
2,2020.01,1,6753,M16w,0-1 (8-7),0,hikiotoshi,12231,M17e,1-0 (11-4),1
3,2020.01,1,12231,M17e,1-0 (11-4),1,hikiotoshi,6753,M16w,0-1 (8-7),0
4,2020.01,1,6473,M16e,1-0 (9-6),1,tsukiotoshi,6491,M15w,0-1 (8-7),0


In [275]:
basho.to_csv('basho.csv', index=False)

# Section 5: Find images of fighters

In [276]:
updated_fighters.head()

,id,stable_id,rank,rikishi,shusshin,birth_date,height,weight
0,1123,36,Y1e,Hakuho,Mongolia,1985-03-11,192.0,151.0
1,1219,30,Y1w,Kakuryu,Mongolia,1985-08-10,186.0,150.0
2,1226,8,M11w,Kotoshogiku,Fukuoka,1984-01-30,180.0,177.0
3,2879,42,M17e,Sadanoumi,Kumamoto,1987-05-11,184.0,131.2
4,5944,12,M10e,Tamawashi,Mongolia,1984-11-16,189.0,168.0


In [283]:
fighter_img = updated_fighters[['id','rikishi']]
fighter_img.head()

,id,rikishi
0,1123,Hakuho
1,1219,Kakuryu
2,1226,Kotoshogiku
3,2879,Sadanoumi
4,5944,Tamawashi


In [284]:
len(fighter_img)

63

In [285]:
# Import API key
from config import gimgkey

In [287]:
#Perform API Call

#Create needed columns to store data for each fighter_image
fighter_img['img_url'] =''

#Perform API call to get information for each stable
base_url = 'https://app.zenserp.com/api/v2/search'

params = {"apikey": gimgkey, 'tbm':'isch','num':'1'}

#Iterate through the list of fighetrs to retrieve their information
for i, row in fighter_img.iterrows():
    
    params['q'] = row['rikishi']

    response = requests.get(base_url,params=params).json()
    
    try:
        fighter_img.loc[i,'img_url']= response['image_results'][0]['sourceUrl']
        
    except:
        #Assign NaN to Fighters not found
        fighter_img.loc[i,'img_url']= np.nan

<ipython-input-287-9d5ea8b57925>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  fighter_img['img_url'] =''
/Users/carlospazos/opt/anaconda3/lib/python3.8/site-packages/pandas/core/indexing.py:966: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


In [291]:
fighter_img.to_csv('fighter_img.csv',index=False)

# Section 6: Write to Postgress